<a href="https://colab.research.google.com/github/Aman78695/Vector-Database_Pinecone/blob/main/Vector_Database_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vector databases are purpose-built to handle the unique structure of vector embeddings. They index vectors for easy search and retrieval by comparing values and finding those that are most similar to one another.

In [9]:
!pip install sentence-transformers

In [10]:
!pip install pinecone-client

In [11]:
import pandas as pd

In [12]:
data=pd.read_csv('/content/df_fe_without_preprocessing_train.csv',on_bad_lines='skip',encoding='latin1')

In [13]:
import pinecone
pinecone.init(api_key='', environment='us-west1-gcp')

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2',device='cuda')

In [ ]:
pinecone.create_index(name='question-search', dimension=768)

In [ ]:

index = pinecone.Index('question-search')

In [ ]:
question_list = []
for i,row in data.iterrows():
  question_list.append(
      (
        str(row['id']),
        model.encode(row['question1']).tolist(),
        {
            'is_duplicate': int(row['is_duplicate']),
            'question1': row['question1']
        }
      )
  )
  if len(question_list)==50 or len(question_list)==len(data):
    index.upsert(vectors=question_list)
    question_list = []

In [ ]:
query = "How do I prepare for civil service?"
xq = model.encode([query]).tolist()
result = index.query(xq, top_k=2, includeMetadata=True)
result